In [1]:
%load_ext autoreload
%autoreload 2

import matplotlib.pyplot as plt

from keras_preprocessing.image import DataFrameIterator
import random
import numpy as np
import pandas as pd
from keras.layers import Dense, Flatten, Input, Reshape
from keras.models import Sequential, Model
from PIL import Image

Using TensorFlow backend.


### Base dataframe

In [2]:
pixel_val = 1
np_filenames, img_filenames = [], []
for i in range(20):
    np_filename = '/tmp/{}.npy'.format(i)
    img_filename = '/tmp/{}.jpg'.format(i)
    array = pixel_val * np.random.randint(0, 255, size=(2, 2)).astype('float32')
    np.save(np_filename, array[..., None])
    plt.imsave(img_filename, array)
    np_filenames.append(np_filename)
    img_filenames.append(img_filename)
    
df = pd.DataFrame({'img_path': img_filenames, 'np_filename': np_filenames}).sample(frac=1).reset_index(drop=True)
df['img_path_extra'] = img_filenames
df['regression'] = np.random.randint(1, 10, size=len(df))
df['regression_extra'] = np.random.randint(10, 100, size=len(df))
df['binary'] = random.sample(['dog', 'cat'] * 30, len(df))
df['multi_class'] = random.sample(['dog', 'cat', 'horse'] * 30, len(df))
df['multi_label'] = random.sample(['dog', 'cat', ['dog'], ['cat'], ['cat', 'dog'], ['dog', 'cat']] * 10, len(df))
print(len(df))
df.head()

20


,img_path,np_filename,img_path_extra,regression,regression_extra,binary,multi_class,multi_label
0,/tmp/14.jpg,/tmp/14.npy,/tmp/0.jpg,3,47,dog,cat,[dog]
1,/tmp/2.jpg,/tmp/2.npy,/tmp/1.jpg,1,27,dog,cat,cat
2,/tmp/8.jpg,/tmp/8.npy,/tmp/2.jpg,4,52,cat,cat,dog
3,/tmp/11.jpg,/tmp/11.npy,/tmp/3.jpg,2,94,cat,cat,cat
4,/tmp/12.jpg,/tmp/12.npy,/tmp/4.jpg,4,95,cat,horse,cat


### Inference

In [3]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
[]
(2, 255, 255, 3)


In [4]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path'],
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
[]
(2, 255, 255, 3)


In [5]:
df_iter = DataFrameIterator(
    df, 
    input_columns=['img_path', 'img_path_extra'],
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
for x in next(df_iter):
    print(x.shape)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}, {'column': 'img_path_extra', 'valid_filepaths': ['/tmp/0.jpg', '/tmp/1.jpg', '/tmp/2.jpg', '/tmp/3.jpg', '/tmp/4.jpg', '/tmp/5.jpg', '/tmp/6.jpg', '/tmp/7.jpg', '/tmp/8.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/13.jpg', '/tmp/14.jpg', '/tmp/15.jpg', '/tmp/16.jpg', '/tmp/17.jpg', '/tmp/18.jpg', '/tmp/19.jpg']}]
[]
(2, 255, 255, 3)
(2, 255, 255, 3)


### Regression

In [6]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns='regression',
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255, 255, 3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
[{'column': 'regression', 'mode': None, 'values': 0     3.0
1     1.0
2     4.0
3     2.0
4     4.0
5     4.0
6     4.0
7     3.0
8     7.0
9     1.0
10    9.0
11    4.0
12    1.0
13    8.0
14    4.0
15    3.0
16    6.0
17    8.0
18    4.0
19    8.0
Name: regression, dtype: float32}]


W1007 08:05:56.628502 140291946174272 deprecation_wrapper.py:119] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/keras/backend/tensorflow_backend.py:422: The name tf.global_variables is deprecated. Please use tf.compat.v1.global_variables instead.



Epoch 1/1
10/10 [==============================] - 0s 13ms/step - loss: 89025463.7638


In [7]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['regression'],
    batch_size=2
)
print(df_iter.inputs)
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(1))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'valid_filepaths': ['/tmp/14.npy', '/tmp/2.npy', '/tmp/8.npy', '/tmp/11.npy', '/tmp/12.npy', '/tmp/7.npy', '/tmp/17.npy', '/tmp/4.npy', '/tmp/16.npy', '/tmp/9.npy', '/tmp/10.npy', '/tmp/3.npy', '/tmp/13.npy', '/tmp/15.npy', '/tmp/19.npy', '/tmp/18.npy', '/tmp/6.npy', '/tmp/5.npy', '/tmp/1.npy', '/tmp/0.npy']}]
[{'column': 'regression', 'mode': None, 'values': 0     3.0
1     1.0
2     4.0
3     2.0
4     4.0
5     4.0
6     4.0
7     3.0
8     7.0
9     1.0
10    9.0
11    4.0
12    1.0
13    8.0
14    4.0
15    3.0
16    6.0
17    8.0
18    4.0
19    8.0
Name: regression, dtype: float32}]
Epoch 1/1
10/10 [==============================] - 0s 10ms/step - loss: 338260042.1687


### Binary classification

In [8]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

W1007 08:05:57.322124 140291946174272 deprecation.py:323] From /home/rodrigo/anaconda3/envs/keras-dev/lib/python3.6/site-packages/tensorflow/python/ops/nn_impl.py:180: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
[{'column': 'binary', 'mode': 'sparse', 'values': array([1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 1.], dtype=float32), 'class_indices': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 12ms/step - loss: 8336.6653


In [9]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    input_color_modes='grayscale',
    input_image_sizes=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)
model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'valid_filepaths': ['/tmp/14.npy', '/tmp/2.npy', '/tmp/8.npy', '/tmp/11.npy', '/tmp/12.npy', '/tmp/7.npy', '/tmp/17.npy', '/tmp/4.npy', '/tmp/16.npy', '/tmp/9.npy', '/tmp/10.npy', '/tmp/3.npy', '/tmp/13.npy', '/tmp/15.npy', '/tmp/19.npy', '/tmp/18.npy', '/tmp/6.npy', '/tmp/5.npy', '/tmp/1.npy', '/tmp/0.npy']}]

[{'column': 'binary', 'mode': 'sparse', 'values': array([1., 1., 0., 0., 0., 0., 0., 1., 0., 1., 0., 0., 1., 0., 0., 1., 0.,
       1., 0., 1.], dtype=float32), 'class_indices': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 10ms/step - loss: 73.4072


### Multi-class classsification

In [10]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    batch_size=2,
    input_color_modes='grayscale',
    input_image_sizes=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
[{'column': 'multi_class', 'mode': 'sparse', 'values': array([0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 2., 1., 2., 0., 2., 2.,
       1., 2., 1.], dtype=float32), 'class_indices': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 8ms/step - loss: 78.9067


In [11]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'sparse'},
    batch_size=2,
    input_color_modes='grayscale',
    input_image_sizes=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(2,2,1)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'sparse_categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'valid_filepaths': ['/tmp/14.npy', '/tmp/2.npy', '/tmp/8.npy', '/tmp/11.npy', '/tmp/12.npy', '/tmp/7.npy', '/tmp/17.npy', '/tmp/4.npy', '/tmp/16.npy', '/tmp/9.npy', '/tmp/10.npy', '/tmp/3.npy', '/tmp/13.npy', '/tmp/15.npy', '/tmp/19.npy', '/tmp/18.npy', '/tmp/6.npy', '/tmp/5.npy', '/tmp/1.npy', '/tmp/0.npy']}]

[{'column': 'multi_class', 'mode': 'sparse', 'values': array([0., 0., 0., 0., 2., 0., 0., 0., 0., 0., 0., 2., 1., 2., 0., 2., 2.,
       1., 2., 1.], dtype=float32), 'class_indices': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 8ms/step - loss: 139.8395


In [12]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]

[{'column': 'multi_class', 'mode': 'categorical', 'values': [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 1], 'class_indices': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 18ms/step - loss: 20052.7314


In [13]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(3, activation='softmax'))
model.compile('adam', 'categorical_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'valid_filepaths': ['/tmp/14.npy', '/tmp/2.npy', '/tmp/8.npy', '/tmp/11.npy', '/tmp/12.npy', '/tmp/7.npy', '/tmp/17.npy', '/tmp/4.npy', '/tmp/16.npy', '/tmp/9.npy', '/tmp/10.npy', '/tmp/3.npy', '/tmp/13.npy', '/tmp/15.npy', '/tmp/19.npy', '/tmp/18.npy', '/tmp/6.npy', '/tmp/5.npy', '/tmp/1.npy', '/tmp/0.npy']}]

[{'column': 'multi_class', 'mode': 'categorical', 'values': [0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 2, 1, 2, 0, 2, 2, 1, 2, 1], 'class_indices': {0: 'cat', 1: 'dog', 2: 'horse'}}]
Epoch 1/1
10/10 [==============================] - 0s 36ms/step - loss: 14840.6102


### Multi-label classification

In [14]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]

[{'column': 'multi_label', 'mode': 'categorical', 'values': [array([1]), 0, 1, 0, 0, array([1, 0]), array([0]), array([0, 1]), array([1]), array([0, 1]), array([0, 1]), array([0]), array([1, 0]), array([1]), array([1]), array([1, 0]), array([0]), array([1, 0]), array([1, 0]), array([1, 0])], 'class_indices': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 21ms/step - loss: 8131.7131


In [15]:
df_iter = DataFrameIterator(
    df, 
    input_columns='np_filename',
    output_columns=['multi_label'],
    output_modes={'multi_label': 'categorical'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255,255,3)))
model.add(Dense(2, activation='sigmoid'))
model.compile('adam', 'binary_crossentropy')
model.fit_generator(df_iter)

[{'column': 'np_filename', 'valid_filepaths': ['/tmp/14.npy', '/tmp/2.npy', '/tmp/8.npy', '/tmp/11.npy', '/tmp/12.npy', '/tmp/7.npy', '/tmp/17.npy', '/tmp/4.npy', '/tmp/16.npy', '/tmp/9.npy', '/tmp/10.npy', '/tmp/3.npy', '/tmp/13.npy', '/tmp/15.npy', '/tmp/19.npy', '/tmp/18.npy', '/tmp/6.npy', '/tmp/5.npy', '/tmp/1.npy', '/tmp/0.npy']}]

[{'column': 'multi_label', 'mode': 'categorical', 'values': [array([1]), 0, 1, 0, 0, array([1, 0]), array([0]), array([0, 1]), array([1]), array([0, 1]), array([0, 1]), array([0]), array([1, 0]), array([1]), array([1]), array([1, 0]), array([0]), array([1, 0]), array([1, 0]), array([1, 0])], 'class_indices': {0: 'cat', 1: 'dog'}}]
Epoch 1/1
10/10 [==============================] - 0s 30ms/step - loss: 6937.0640


### Output same as input

In [24]:
df_iter = DataFrameIterator(
    df, 
    input_columns='img_path',
    output_columns=['img_path'],
    output_modes={'img_path': 'image'},
    batch_size=2,
    shuffle=False
)
print(df_iter.inputs)
print()
print(df_iter.outputs)

model = Sequential()
model.add(Flatten(input_shape=(255, 255, 3)))
model.add(Dense(100, activation='sigmoid'))
model.add(Dense(255 * 255 * 3, activation='sigmoid'))
model.add(Reshape((255, 255, 3)))
model.compile('adam', 'mse')
model.fit_generator(df_iter)

[{'column': 'img_path', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]

[{'column': 'img_path', 'mode': 'image', 'valid_filepaths': ['/tmp/14.jpg', '/tmp/2.jpg', '/tmp/8.jpg', '/tmp/11.jpg', '/tmp/12.jpg', '/tmp/7.jpg', '/tmp/17.jpg', '/tmp/4.jpg', '/tmp/16.jpg', '/tmp/9.jpg', '/tmp/10.jpg', '/tmp/3.jpg', '/tmp/13.jpg', '/tmp/15.jpg', '/tmp/19.jpg', '/tmp/18.jpg', '/tmp/6.jpg', '/tmp/5.jpg', '/tmp/1.jpg', '/tmp/0.jpg']}]
Epoch 1/1
10/10 [==============================] - 6s 554ms/step - loss: 17973.9524


### Multi-output: regression  & regression

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='img_filename',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='np_filename',
    output_columns=['regression', 'regression_extra'],
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1)(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', 'mse')
model.fit_generator(df_iter)

### Multi-output: regression & binary classification

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='img_filename',
    output_columns=['regression', 'binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'binary_crossentropy'])
model.fit_generator(df_iter)

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='np_filename',
    output_columns=['regression', 'binary'],
    output_modes={'binary': 'sparse'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(1, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'binary_crossentropy'])
model.fit_generator(df_iter)

### Multi-output: regression & multi-class classification

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='img_filename',
    output_columns=['regression', 'multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(3, activation='softmax')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'categorical_crossentropy'])
model.fit_generator(df_iter)

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='np_filename',
    output_columns=['regression', 'multi_class'],
    output_modes={'multi_class': 'categorical'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(1)(x)
out_2 = Dense(3, activation='softmax')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['mse', 'categorical_crossentropy'])
model.fit_generator(df_iter)

### Multi-output: multi-class & multi-label classification

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='img_filename',
    output_columns=['multi_class', 'multi_label'],
    output_modes={'multi_class': 'categorical', 'multi_label': 'categorical'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(3, activation='softmax')(x)
out_2 = Dense(2, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['categorical_crossentropy', 'binary_crossentropy'])
model.fit_generator(df_iter)

In [ ]:
df_iter = DataFrameIterator(
    df, 
    input_column='np_filename',
    output_columns=['multi_class', 'multi_label'],
    output_modes={'multi_class': 'categorical', 'multi_label': 'categorical'},
    batch_size=2,
    color_mode='grayscale',
    target_size=(2,2),
    shuffle=False
)
print(df_iter.inputs)
print(df_iter.outputs)

_input = Input(shape=(2, 2, 1))
x = Flatten()(_input)
out_1 = Dense(3, activation='softmax')(x)
out_2 = Dense(2, activation='sigmoid')(x)
model = Model(_input, [out_1, out_2])
model.compile('adam', loss=['categorical_crossentropy', 'binary_crossentropy'])
model.fit_generator(df_iter)